Required Librarries to be installed

In [ ]:
!pip install rdkit-pypi
!pip install ace_tools
!pip install catboost
!pip install pandas numpy scikit-learn xgboost catboost seaborn matplotlib tensorflow rdkit
!pip install tensorflow==2.12.0

Import libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer
from scipy.stats import pearsonr
from xgboost import XGBRegressor
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.base import BaseEstimator, RegressorMixin
import pickle
from sklearn.model_selection import cross_val_score
import os
import scipy.stats as stats

Load your dataset: CSV format

In [ ]:
data = pd.read_csv(your_data)

In [ ]:
print(data.head())

Functions to convert SMILES to Molecular Fingerprints and Descriptors

In [ ]:
# Function to convert SMILES to molecular fingerprint
def smiles_to_fingerprint(smiles, radius=2, n_bits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        return np.array(fp)
    else:
        return np.zeros(n_bits)

# Function to calculate RDKit descriptors
def calculate_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        return np.array([desc[1](mol) for desc in Descriptors.descList])
    else:
        return np.zeros(len(Descriptors.descList))

Convert SMILES to fingerprints and descriptors

In [ ]:
data['Fingerprint'] = data['SMILES'].apply(smiles_to_fingerprint)
data['Descriptors'] = data['SMILES'].apply(calculate_descriptors)

Combine fingerprints and descriptors into one feature matrix and concatenate them in x and put target values (Molecular toxicity or property) that you want to predict.

In [ ]:
fingerprints = np.array(data['Fingerprint'].tolist())
descriptors = np.array(data['Descriptors'].tolist())
X = np.concatenate([fingerprints, descriptors], axis=1)
y = data['Experimental logEC50'].values

Check the shapes of fingerprints

In [ ]:
fingerprints.shape

Check the shapes of descriptors

In [ ]:
descriptors.shape

Sometimes fingerprints and descriptors can have missing values because of the limitations of RDkit. so, first check if there is any missing values,

In [ ]:
# Count the total number of missing (NaN) values in X
missing_values_count = np.isnan(X).sum()

print("Number of missing values in X:", missing_values_count)

If you have missing values, you can handle missing values by imputing with the mean value or other methods. Here, we implemented the mean for handling the missing values.

In [ ]:
# Handle missing values by imputing with the mean value
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

Check the shape of the X. the columns numbers must be as same as sum of fingerprints and descriptors.

In [ ]:
X.shape

you can keep the name of features with the following code, if you want to have an interpretability study on the most important features.

In [ ]:
# Define feature names
fingerprint_features = [f'fingerprint_{i}' for i in range(fingerprints.shape[1])]
descriptor_features = [desc[0] for desc in Descriptors.descList]
feature_names = fingerprint_features + descriptor_features

In [ ]:
# Ensure the length of feature_names matches the number of columns in X
assert len(feature_names) == X.shape[1], "Length of feature_names must match number of columns in X"

In this part, we are going to initialize feature selection process by using Random Forest and Recursive Feature Elimination (RFE)

In [ ]:
# Initialize Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Define a range of features to evaluate
# Since we have 2,258 features, we can start with larger steps initially and then narrow down
feature_range = list(range(50, 1001, 50)) + list(range(1000, 2001, 200)) + [2258]

scores = []

# Perform RFE for each number of features in the range
for n_features in feature_range:
    selector = RFE(estimator=rf, n_features_to_select=n_features, step=50)
    score = cross_val_score(selector, X, y, cv=5, scoring='r2').mean()  # Using R² as a performance metric
    scores.append(score)

In [ ]:
# Find the optimal number of features
optimal_features = feature_range[np.argmax(scores)]
print(f'Optimal number of features: {optimal_features}')


To find the optimal number of features, we implemented "Elbow Method"

In [ ]:
# Plot the scores to find the optimal number of features
plt.figure(figsize=(10, 6))
plt.plot(feature_range, scores, marker='o')
plt.xlabel('Number of Features')
plt.ylabel('Cross-Validation R² Score')
plt.title('Elbow Method for Optimal Number of Features')
plt.grid(True)
plt.show()

# Find the optimal number of features
optimal_features = feature_range[np.argmax(scores)]
print(f'Optimal number of features: {optimal_features}')

Refit the RFE selctor with the optimal number of features

In [ ]:
selector = RFE(estimator=rf, n_features_to_select=optimal_features, step=50)
selector = selector.fit(X, y)

Transform data to select the optimal features

In [ ]:
X_selected = selector.transform(X)
print(f'Shape of the new feature set: {X_selected.shape}')

if you want to get the ranking of the features from the RFE selector run the code below. This code gives you the most important features names.

In [ ]:
# Get the ranking of the features from the RFE selector
ranking = selector.ranking_

# Identify the important features (those with a ranking of 1)
important_indices = np.where(ranking == 1)[0]
important_features = [feature_names[i] for i in important_indices]

# Get the feature importances from the RandomForest model
importances = selector.estimator_.feature_importances_

# Sort the features by importance
sorted_indices = np.argsort(importances)[::-1]
sorted_features = [important_features[i] for i in sorted_indices]
sorted_importances = importances[sorted_indices]

# Plot the top N important features
N = 20  # Number of top features to display
plt.figure(figsize=(10, 8))
plt.barh(sorted_features[:N], sorted_importances[:N], color='skyblue')
plt.xlabel('Feature Importance')
plt.ylabel('Feature Name')
plt.title(f'Top {N} Important Features')
plt.gca().invert_yaxis()  # To have the most important feature at the top
plt.show()

Split the data into training and testing sets and check the shapes of each set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

Training base models (RF: random forest, SVR: support vector machine, catboost, Chemception)

Here, we are going to use GridsearchCV for finding the best hyperparameters.

In [ ]:
# Hyperparameter grids for base models
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

param_grid_svr = {
    'kernel': ['rbf'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto']
}

param_grid_catboost = {
    'depth': [6, 8],
    'learning_rate': [0.1, 0.01],
    'iterations': [100, 200]
}


In [ ]:
# GridSearch for hyperparameter tuning
grid_search_rf = GridSearchCV(RandomForestRegressor(random_state=42), param_grid_rf, cv=5, n_jobs=-1, verbose=1)
grid_search_svr = GridSearchCV(SVR(), param_grid_svr, cv=5, n_jobs=-1, verbose=1)
grid_search_catboost = GridSearchCV(CatBoostRegressor(random_state=42, silent=True), param_grid_catboost, cv=5, n_jobs=-1, verbose=1)

grid_search_rf.fit(X_train, y_train)
grid_search_svr.fit(X_train, y_train)
grid_search_catboost.fit(X_train, y_train)

best_rf = grid_search_rf.best_estimator_
best_svr = grid_search_svr.best_estimator_
best_catboost = grid_search_catboost.best_estimator_

defining the Chemception model

In [ ]:
# Chemception model definition
def create_chemception_model(input_shape, filters=32, kernel_size=3, pool_size=2, dense_units=100, learning_rate=0.001):
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

In [ ]:
# Wrap Chemception model for hyperparameter tuning
chemception = KerasRegressor(build_fn=create_chemception_model, input_shape=(X_train.shape[1], 1))

In [ ]:
# Hyperparameter grid for Chemception
param_distribs_chemception = {
    'filters': [32, 64],
    'kernel_size': [3, 5],
    'pool_size': [2, 3],
    'dense_units': [50, 100],
    'epochs': [10, 20],
    'batch_size': [10, 20],
    'learning_rate': [0.001, 0.01]
}

In [ ]:
# RandomizedSearchCV for Chemception
random_search_chemception = RandomizedSearchCV(chemception, param_distribs_chemception, n_iter=10, cv=3, n_jobs=-1, verbose=1)
random_search_chemception.fit(X_train.reshape(-1, X_train.shape[1], 1), y_train)

best_chemception = random_search_chemception.best_estimator_

If you want to see the total parameters for best_chemception mode, you can use this code.

In [ ]:
model = best_chemception.model
model.summary()

Here, we are going to generate cross-validated predictions for each base model and create meta-features.

In [ ]:
meta_features = np.zeros((X_train.shape[0], 4))
meta_features[:, 0] = cross_val_predict(best_rf, X_train, y_train, cv=5)
meta_features[:, 1] = cross_val_predict(best_svr, X_train, y_train, cv=5)
meta_features[:, 2] = cross_val_predict(best_catboost, X_train, y_train, cv=5)

In [ ]:
# Ensure predictions are reshaped correctly
chemception_predictions = cross_val_predict(best_chemception, X_train.reshape(-1, X_train.shape[1], 1), y_train, cv=5)
meta_features[:, 3] = chemception_predictions.reshape(-1)

now, after having meta-feature, we are going to initialize the meta-model which is XGBoost in this study.

In [ ]:
# Hyperparameter grid for meta-model (XGBoost)
param_grid_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.02],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

# GridSearchCV for meta-model
grid_search_xgb = GridSearchCV(XGBRegressor(random_state=42), param_grid_xgb, cv=5, n_jobs=-1, verbose=1)
grid_search_xgb.fit(meta_features, y_train)

best_xgb = grid_search_xgb.best_estimator_

then, you have to generate the same features for test dataset.

In [ ]:
test_meta_features = np.zeros((X_test.shape[0], 4))
test_meta_features[:, 0] = best_rf.predict(X_test)
test_meta_features[:, 1] = best_svr.predict(X_test)
test_meta_features[:, 2] = best_catboost.predict(X_test)
# Ensure predictions are reshaped correctly
chemception_test_predictions = best_chemception.predict(X_test.reshape(-1, X_test.shape[1], 1))
test_meta_features[:, 3] = chemception_test_predictions.reshape(-1)

Finally, we are going to do the final prediction by using meta-model which has been trained based on the best hyperparameters which GridsearchCV found.

In [ ]:
# Make final predictions
final_predictions = best_xgb.predict(test_meta_features)

Evaluate the performance with difference metrics.

In [ ]:
mse = mean_squared_error(y_test, final_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, final_predictions)
r2 = r2_score(y_test, final_predictions)
pearson_corr, _ = pearsonr(y_test, final_predictions)

# Print the metrics
metrics = {
    'RMSE': rmse,
    'MAE': mae,
    'R-squared': r2,
    'Pearson Correlation': pearson_corr
}

# Display the metrics
metrics_df = pd.DataFrame.from_dict(metrics, orient='index', columns=['Value'])
print(metrics_df)

In [ ]:
# Scatter plot for predicted vs actual values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, final_predictions, alpha=0.6, color='b')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual Experimental logEC50')
plt.ylabel('Predicted Experimental logEC50')
plt.title('Actual vs Predicted Experimental logEC50')
plt.show()

# Residuals plot
residuals = y_test - final_predictions
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, color='r')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Residuals Distribution')
plt.show()

calculate the Standard Deviatin of Errors.

In [ ]:
errors = y_test - final_predictions
std_dev = np.std(errors)
print(f'Standard Deviation of Errors: {std_dev}')

Calculate the mean and standard error and  Calculate the 95% confidence interval

In [ ]:
# Calculate the mean and standard error
mean_pred = np.mean(final_predictions)
std_dev = np.std(final_predictions, ddof=1)  # Use ddof=1 for sample standard deviation
n = len(final_predictions)
std_err = std_dev / np.sqrt(n)

# Calculate the 95% confidence interval
confidence_level = 0.95
degrees_freedom = n - 1
confidence_interval = stats.t.interval(confidence_level, degrees_freedom, mean_pred, std_err)


print(f"Mean prediction: {mean_pred:.4f}")
print(f"95% Confidence Interval: {confidence_interval}")

Below, you can calculate standard deviation and mean for the predictions and the 95% confidence interval for each point and confidence intervals for each point.

In [ ]:
# Calculate standard deviation and mean for the predictions
mean_pred = np.mean(final_predictions)
std_dev = np.std(final_predictions, ddof=1)

# Calculate the 95% confidence interval for each point
n = len(final_predictions)
confidence_interval = stats.t.interval(0.95, n-1, loc=mean_pred, scale=std_dev/np.sqrt(n))

# Calculate confidence intervals for each point
confidence_intervals = []
for pred in final_predictions:
    ci_low = pred - 1.96 * (std_dev / np.sqrt(n))
    ci_high = pred + 1.96 * (std_dev / np.sqrt(n))
    confidence_intervals.append((ci_low, ci_high))

# Print the first few confidence intervals
print(confidence_intervals[:5])

Calculate t_test and P-value

In [ ]:
from scipy.stats import ttest_rel

# Perform a paired t-test
t_stat, p_value = ttest_rel(final_predictions, y_test)

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

Print values to inspect

In [ ]:
print("Predicted values:", final_predictions)
print("Actual values:", y_test)

In [ ]:
differences = np.abs(final_predictions - y_test)
print("Absolute differences:", differences)

Calculate absolute differences and plot them to analyze visually

In [ ]:
# Calculate absolute differences
differences = np.abs(final_predictions - y_test)
print("Absolute differences:", differences)

# Plot the differences
plt.figure(figsize=(10, 6))
x = np.arange(1, len(differences) + 1)
plt.plot(x, differences, 'o-', label='Absolute Difference')

# Add labels and title
plt.xlabel('Data Point')
plt.ylabel('Absolute Difference')
plt.title('Absolute Differences Between Predicted and Actual Values')
plt.legend()
plt.show()

Generate synthetic p-values for demonstration

In [ ]:
# Generate synthetic p-values for demonstration
np.random.seed(42)
p_values = np.random.uniform(0, 0.1, len(final_predictions))  # Simulating some p-values for illustration

# Set up the plot
x = np.arange(1, len(p_values) + 1)
significance_level = 0.05

plt.figure(figsize=(10, 6))

# Plot the p-values
plt.plot(x, p_values, 'o-', label='p-value')

# Add shading for significant p-values (p < 0.05)
plt.fill_between(x, 0, significance_level, where=p_values < significance_level, color='red', alpha=0.3, label='Significant (p < 0.05)')

# Add a horizontal line at the significance level
plt.axhline(y=significance_level, color='gray', linestyle='--', label='Significance level (0.05)')

# Labels and title
plt.xlabel('Data Point')
plt.ylabel('p-value')
plt.title('P-values with Shaded Significance Area')
plt.legend()

# Show the plot
plt.show()